In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "/share/dutta/eyao/dataset/kaggle/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/share/dutta/eyao/dataset/kaggle'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory () that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Importing Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
import xgboost as xgb
from lightgbm import LGBMClassifier
import lightgbm as lgb
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, log_loss
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

from sklearn.preprocessing import LabelEncoder

# Training and Testing Data

In [ ]:
train_data = pd.read_csv('/share/dutta/eyao/dataset/kaggle/spaceship-titanic/train.csv')
test_data = pd.read_csv('/share/dutta/eyao/dataset/kaggle/spaceship-titanic/test.csv')
sample_submission_data = pd.read_csv('/share/dutta/eyao/dataset/kaggle/spaceship-titanic/sample_submission.csv')

In [ ]:
train_data.head()

In [ ]:
test_data.head()

In [ ]:
train_data.info()

In [ ]:
train_data.isnull().sum()

In [ ]:
train_data.shape

In [ ]:
train_data.drop_duplicates(inplace = True)

In [ ]:
train_data.shape

#### No duplicates

In [ ]:
train_data.describe()

# Visualising data

In [ ]:
train_data['Age'].hist(figsize=(5,3), color='g')

#### Majority of the people belong to the age group 15-25

In [ ]:
train_data['HomePlanet'].value_counts()
sns.countplot(x = 'HomePlanet', hue = 'Transported' ,data = train_data)

Transported means that the person was transported to another dimension

In [ ]:
train_data['CryoSleep'].value_counts()
sns.countplot(x='CryoSleep', hue='Transported',data=train_data)

In [ ]:
train_data['Age'].mode()[0]

In [ ]:
train_data['Age'].median()

# Handling Missing Values

In [ ]:
for col_name in train_data.columns:
    if train_data[col_name].dtypes=='object':
        train_data[col_name] = train_data[col_name].fillna(train_data[col_name].mode()[0])
    else:
        train_data[col_name] = train_data[col_name].fillna(train_data[col_name].median())
print(train_data.shape)

In [ ]:
for col_name in test_data.columns:
    if test_data[col_name].dtypes=='object':
        test_data[col_name] = test_data[col_name].fillna(test_data[col_name].mode()[0])
    else:
        test_data[col_name] = test_data[col_name].fillna(test_data[col_name].median())
print(test_data.shape)

# Encoding Values

In [ ]:
encoder = LabelEncoder()
for col_name in train_data.columns:
    if train_data[col_name].dtypes == 'object':
        train_data[col_name] = encoder.fit_transform(train_data[col_name])

In [ ]:
object_columns = test_data.select_dtypes(include='object').columns.difference(['PassengerId'])
encoder = LabelEncoder()
for col_name in object_columns:
    if test_data[col_name].dtype == 'object':
        test_data[col_name] = encoder.fit_transform(test_data[col_name])

In [ ]:
plt.figure(figsize=(18,12))
sns.heatmap(train_data.corr(), annot=True, cmap='coolwarm', fmt=".2f")

# Building the Model

In [ ]:
X_train = train_data.drop(['Transported','PassengerId','Name','ShoppingMall'], axis=1)
y_train = train_data['Transported']
X_test = test_data.drop(['PassengerId','Name','ShoppingMall'], axis=1)

In [ ]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Using XGBoost

In [ ]:
model_XGB = XGBClassifier(learning_rate = 0.1 , max_depth = 4, n_estimators = 100)
model_XGB.fit(X_train, y_train)
XGB_pred = model_XGB.predict(X_test)

### HyperParameter Tuning

In [ ]:
# param_grid = {
#     'learning_rate': [0.01, 0.1, 0.2],
#     'max_depth': [3, 4, 5, 7, 8, 10],
#     'n_estimators': [50, 100, 150, 200]
# }

In [ ]:
# grid_search = GridSearchCV(estimator = model_XGB,param_grid = param_grid, cv = 5, scoring = 'roc_auc' )

In [ ]:
# grid_search.fit(X_train, y_train)

In [ ]:
# best_params = grid_search.best_params_
# print("Best Hyperparameters:", best_params)

# Final Submission

In [ ]:
submission = pd.DataFrame({
    'PassengerId': test_data.PassengerId,
    'Transported': XGB_pred
})
submission['Transported'] = submission['Transported'].astype(bool)
print(submission.head())

In [ ]:
submission.to_csv('submission.csv', index = False)